In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit

In [55]:
def back_testing(df,model,predictors,start=1,step=36):
    all_predictions = []
    print(range(start,df.shape[0],step))
    for i in range(start,df.shape[0],step):
        train=df.iloc[:i,:]
        print(train)
        
        test=df.iloc[i:i+step,:]
        print('--------test------')
        print(test)
        model.fit(train[predictors],train["STATUS"])

        preds = model.predict(test[predictors])

        preds = pd.Series(preds , index=test.index)
        combined = pd.concat([test["STATUS"],preds], axis=1)
        combined.columns = ["actual","prediction"]
        print('------ pres')
        print(combined)
        
        #print(all_predictions,combined)  
        all_predictions.append(combined)
        print('---------------------------')
   
    alpha = pd.concat(all_predictions) 
    return  alpha

In [2]:
df=pd.read_excel(r'C:\Users\hp\Desktop\M2\PFE\Code\code pfe\Coud source\Code\data\Well-26_06-09-2020.xlsx')

In [43]:
df1=df[['TVA (m3)', 'SPPA (kPa)', 'MFOA (m3/s)',
       'MFOP ((m3/s)/(m3/s))', 'GASA (mol/mol)', 'STATUS']]

In [50]:
predictors=df1.columns[df1.columns.isin(['TVA (m3)', 'SPPA (kPa)', 'MFOA (m3/s)',
       'MFOP ((m3/s)/(m3/s))', 'GASA (mol/mol)', 'STATUS'])]
predictors

Index(['TVA (m3)', 'SPPA (kPa)', 'MFOA (m3/s)', 'MFOP ((m3/s)/(m3/s))',
       'GASA (mol/mol)', 'STATUS'],
      dtype='object')

In [56]:
knn = KNeighborsClassifier(n_neighbors=5)
res=back_testing(df1,knn,predictors,start=5,step=36)

range(5, 16374, 36)
    TVA (m3)   SPPA (kPa)  MFOA (m3/s)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)  \
0  82.519116  9245.971389     0.166667              0.175884             0.0   
1  82.464060  9191.947392     0.166667              0.192889             0.0   
2  82.448670  9216.589391     0.166667              0.196450             0.0   
3  82.479989  9217.697391     0.166667              0.189825             0.0   
4  82.564425  9149.898395     0.166667              0.186856             0.0   

   STATUS  
0       0  
1       0  
2       0  
3       0  
4       0  
--------test------
     TVA (m3)   SPPA (kPa)  MFOA (m3/s)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)  \
5   82.499055  8985.790406     0.166667              0.190296             0.0   
6   82.355204  9015.457404     0.166667              0.178971             0.0   
7   82.400702  9014.815404     0.166667              0.186028             0.0   
8   82.517086  9078.120400     0.166667              0.185040             0.0   
9  

In [33]:
predict=res[res["actual"]==res['prediction']].sum()
cm = confusion_matrix(res["actual"], res['prediction'])
precision = precision_score(res["actual"], res['prediction'])
recall = recall_score(res["actual"], res['prediction'])
f1 = f1_score(res["actual"], res['prediction'])
accuracy = accuracy_score(res["actual"], res['prediction'])


In [34]:

print('Confusion Matrix:\n', cm)
print('Precision:', precision)
print('Recall:', recall)
print('F1-Score:',f1)
print('Accuracy:',accuracy)
print('-----------------------')

Confusion Matrix:
 [[15918    33]
 [  148   270]]
Precision: 0.8910891089108911
Recall: 0.645933014354067
F1-Score: 0.7489597780859917
Accuracy: 0.9889425132873114
-----------------------


In [57]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit

df=pd.read_excel(r'C:\Users\hp\Desktop\M2\PFE\Code\code pfe\Coud source\Code\data\Well-26_06-09-2020.xlsx')



In [69]:
X = df[['TVA (m3)', 'SPPA (kPa)', 'MFOP ((m3/s)/(m3/s))', 'GASA (mol/mol)']]
y = df['STATUS'].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [70]:
X

,TVA (m3),SPPA (kPa),MFOP ((m3/s)/(m3/s)),GASA (mol/mol)
0,82.519116,9245.971389,0.175884,0.000000
1,82.464060,9191.947392,0.192889,0.000000
2,82.448670,9216.589391,0.196450,0.000000
3,82.479989,9217.697391,0.189825,0.000000
4,82.564425,9149.898395,0.186856,0.000000
...,...,...,...,...
16369,80.338498,6236.098088,0.237633,0.021669
16370,80.315214,6230.651088,0.231971,0.021639
16371,80.303256,6230.454588,0.268173,0.021639
16372,80.532201,6280.804085,0.256460,0.021632


In [72]:
n_splits=36
# Initialize lists to store evaluation metrics
confusion_matrices = []
precisions = []
recalls = []
f1_scores = []
accuracies = []
tscv = TimeSeriesSplit(n_splits=n_splits)
# Iterate through each split
for train_index, test_index in tscv.split(X,y):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index],  X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print('---------TRAIN---------')
    print(X_train)
    print('--------TEST----------')
    print(X_test)
    print(set(y_train),set(y_test))
    model = KNeighborsClassifier(n_neighbors=3)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Compute evaluation metrics
    cm = confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    # Append the metrics to the respective lists
    confusion_matrices.append(cm)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    accuracies.append(accuracy)

---------TRAIN---------
      TVA (m3)   SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
0    82.519116  9245.971389              0.175884             0.0
1    82.464060  9191.947392              0.192889             0.0
2    82.448670  9216.589391              0.196450             0.0
3    82.479989  9217.697391              0.189825             0.0
4    82.564425  9149.898395              0.186856             0.0
..         ...          ...                   ...             ...
457  82.519714  8869.763414              0.202120             0.0
458  82.427288  8898.902412              0.166780             0.0
459  82.478800  8986.286406              0.169179             0.0
460  82.431704  8947.244408              0.172239             0.0
461  82.487309  8971.900407              0.177516             0.0

[462 rows x 4 columns]
--------TEST----------
      TVA (m3)    SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
462  82.454644   9012.190404              0.204060             0.0
463

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\hp\AppData\L

---------TRAIN---------
       TVA (m3)    SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
0     82.519116   9245.971389              0.175884             0.0
1     82.464060   9191.947392              0.192889             0.0
2     82.448670   9216.589391              0.196450             0.0
3     82.479989   9217.697391              0.189825             0.0
4     82.564425   9149.898395              0.186856             0.0
...         ...           ...                   ...             ...
1341  91.258720  13871.950083              0.160801             0.0
1342  91.274553  13811.750087              0.151456             0.0
1343  91.192389  13837.286085              0.184115             0.0
1344  91.125591  13804.132087              0.157403             0.0
1345  91.049831  13783.264089              0.159969             0.0

[1346 rows x 4 columns]
--------TEST----------
       TVA (m3)    SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
1346  90.964161  13769.348090              0

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\hp\AppData\L

---------TRAIN---------
       TVA (m3)   SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
0     82.519116  9245.971389              0.175884             0.0
1     82.464060  9191.947392              0.192889             0.0
2     82.448670  9216.589391              0.196450             0.0
3     82.479989  9217.697391              0.189825             0.0
4     82.564425  9149.898395              0.186856             0.0
...         ...          ...                   ...             ...
2225  86.550572  7361.045513              0.343715             0.0
2226  86.556190  7369.660513              0.332349             0.0
2227  86.591696  7405.890010              0.339543             0.0
2228  86.578937  7361.276513              0.320682             0.0
2229  86.679874  7418.926010              0.323766             0.0

[2230 rows x 4 columns]
--------TEST----------
       TVA (m3)   SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
2230  86.667802  7358.003514              0.353268       

...

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\hp\AppData\L

---------TRAIN---------
        TVA (m3)   SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
0      82.519116  9245.971389              0.175884        0.000000
1      82.464060  9191.947392              0.192889        0.000000
2      82.448670  9216.589391              0.196450        0.000000
3      82.479989  9217.697391              0.189825        0.000000
4      82.564425  9149.898395              0.186856        0.000000
...          ...          ...                   ...             ...
14601  65.000203  3178.689540              0.163426        0.019073
14602  65.023998  3191.789039              0.187739        0.019482
14603  64.971766  3221.756037              0.165225        0.019482
14604  64.989008  3198.359039              0.178958        0.019482
14605  64.942703  3162.558041              0.184870        0.019482

[14606 rows x 4 columns]
--------TEST----------
        TVA (m3)   SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
14606  64.960176  3222.588037              

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\hp\AppData\L

---------TRAIN---------
        TVA (m3)   SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
0      82.519116  9245.971389              0.175884        0.000000
1      82.464060  9191.947392              0.192889        0.000000
2      82.448670  9216.589391              0.196450        0.000000
3      82.479989  9217.697391              0.189825        0.000000
4      82.564425  9149.898395              0.186856        0.000000
...          ...          ...                   ...             ...
15485  66.930167  3673.570257              0.261858        0.030122
15486  66.870461  3628.871260              0.251538        0.030122
15487  66.857676  3615.992511              0.256475        0.033633
15488  66.833899  3630.706010              0.252621        0.033633
15489  66.823965  3593.420512              0.256917        0.033633

[15490 rows x 4 columns]
--------TEST----------
        TVA (m3)   SPPA (kPa)  MFOP ((m3/s)/(m3/s))  GASA (mol/mol)
15490  66.802568  3617.483011              

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [62]:
aggregate_cm = sum(confusion_matrices)

print('Confusion Matrix:\n', aggregate_cm)
print('Precision:', sum(precisions)/n_splits)
print('Recall:', sum(recalls)/10)
print('F1-Score:', sum(f1_scores)/n_splits)
print('Accuracy:', sum(accuracies)/n_splits)
print('-----------------------')

Confusion Matrix:
 [[13219  8195]
 [ 8605  8187]]
Precision: 0.0
Recall: 0.0
F1-Score: 0.0
Accuracy: 0.9687797728105533
-----------------------
